In [85]:
'''
Mulliken population analysis with IAO orbitals
'''

import numpy
from functools import reduce
from sys import stdout
from pyscf import gto, scf, lo
from pyscf.tools.dump_mat import dump_rec
from pyscf.tools.dump_mat import dump_mo

h2o_1 = gto.M(atom="""
  O    1.3739407   -0.0638029   -0.2288247
  H    2.1291628    0.5081390   -0.0925662
  H    0.8631284   -0.0251761    0.5847385
""", basis='631g*')

h2o_2 = gto.M(atom="""
  O   -1.3716754   -0.0662078    0.2279844
  H   -2.1274466    0.5076811    0.1037095
  H   -0.8693669   -0.0228122   -0.5906583
""", basis='631g*')

mol = h2o_1 + h2o_2

mf = scf.RHF(mol).run()

orb  = mf.mo_coeff[:,mf.mo_occ>0]
c_ao_lo = lo.iao.iao(mol, orb)
c_ao_lo = lo.vec_lowdin(c_ao_lo, mf.get_ovlp())

converged SCF energy = -152.025489053146


In [105]:
# transform mo_occ to IAO representation. Note the AO dimension is reduced
ovlp_ao = mf.get_ovlp()
dm_ao = mf.make_rdm1()
dm_lo = reduce(numpy.dot, [c_ao_lo.T, ovlp_ao, dm_ao, ovlp_ao, c_ao_lo])
assert numpy.linalg.norm(reduce(numpy.dot, [dm_lo/2, dm_lo/2]) - dm_lo/2) < 1e-8

idx_a = slice(0,7)
idx_b = slice(7,14)

dm_aa = dm_lo[idx_a, idx_a]
dm_bb = dm_lo[idx_b, idx_b]
dm_ab = dm_lo[idx_a, idx_b]

In [89]:
from scipy import linalg

uab, sab, vhab = linalg.svd(dm_ab)
numpy.linalg.norm(
    dm_ab - reduce(numpy.dot, [uab, numpy.diag(sab), vhab])
)

        #0        #1        #2        #3        #4       
0      -0.03381   0.02127   0.05897  -0.02555  -0.94426
1      -0.33700   0.43386   0.25807  -0.25592   0.03946
2       0.20791  -0.63717   0.59574  -0.42171   0.00131
3      -0.03807   0.13767  -0.11927  -0.27980  -0.29444
4      -0.39573  -0.60436  -0.57591   0.01247  -0.05313
5       0.11057  -0.12596   0.30664   0.81490  -0.12334
6       0.81963   0.07248  -0.36754  -0.11622  -0.04575
        #5        #6       
0      -0.29146  -0.13307
1      -0.29748   0.68997
2       0.02216   0.13244
3       0.86570   0.22575
4      -0.10926   0.36271
5       0.18256   0.40655
6      -0.17713   0.37536
        #0        #1        #2        #3        #4       
0       0.02149   0.43512   0.64165   0.13296   0.60019
1      -0.03384  -0.33728  -0.20326  -0.04086   0.39775
2       0.02551   0.25547  -0.42134   0.28016   0.01941
3      -0.05945  -0.26223   0.59257   0.12666  -0.57660
4      -0.16093  -0.37247  -0.00853   0.86262   0.15068
5 

In [107]:
from pyscf import tools

ua, sa, vha = linalg.svd(dm_aa)
numpy.linalg.norm(
    dm_aa - reduce(numpy.dot, [ua, numpy.diag(sa), vha])
)

# print(sa)
# dump_rec(stdout, uab)
# dump_rec(stdout, ua)

ub, sb, vhb = linalg.svd(dm_bb)
numpy.linalg.norm(
    dm_bb - reduce(numpy.dot, [ub, numpy.diag(sb), vhb])
)

# print(sb)
# dump_rec(stdout, ub)
# dump_rec(stdout, vhab.T)

1.407527803117514e-14

In [108]:
c_ao_lo_a = c_ao_lo[:,idx_a]
c_pio_a   = reduce(numpy.dot, [c_ao_lo_a, ua])
npio = c_ao_lo_a.shape[1]

for pp in range(npio):
    tools.cubegen.orbital(mol, f'./cube/h2o_a_{pp}_{s[pp]:4.2f}.cube', c_pio_a[:,pp])

c_ao_lo_b = c_ao_lo[:,idx_b]
c_pio_b   = reduce(numpy.dot, [c_ao_lo_b, ub])
npio = c_ao_lo_b.shape[1]

for pp in range(npio):
    tools.cubegen.orbital(mol, f'./cube/h2o_b_{pp}_{s[pp]:4.2f}.cube', c_pio_b[:,pp])

In [117]:
assert numpy.linalg.norm(
    reduce(numpy.dot, [c_pio_a.T, ovlp_ao, c_pio_a]) - numpy.eye(c_pio_a.shape[1])
)
dump_mo(mol, c_pio_a)

               #0        #1        #2        #3        #4       
  0 O 1s       0.72586  -0.00740  -0.08402  -0.68883   0.01814
  0 O 2s       0.01272   0.51437   0.16544   0.16388   0.09702
  0 O 3s       0.01058   0.50670   0.21155   0.22241   0.17342
  0 O 2px     -0.52246   0.09682   0.05437  -0.56165  -0.03409
  0 O 2py      0.00144   0.16863   0.43287  -0.11978  -0.41489
  0 O 2pz      0.06328   0.38734  -0.36595   0.19712  -0.40143
  0 O 3px     -0.35108   0.06701  -0.01337  -0.36494  -0.01670
  0 O 3py      0.03202   0.11668   0.32641  -0.05747  -0.31644
  0 O 3pz      0.01836   0.27620  -0.26338   0.11322  -0.29339
  0 O 3dxy    -0.01427   0.02029   0.00389  -0.01231   0.00341
  0 O 3dyz     0.00181   0.01151   0.00041  -0.00242  -0.02078
  0 O 3dz^2    0.00536  -0.00092  -0.03339   0.00680  -0.01550
  0 O 3dxz    -0.01871   0.00716   0.00268  -0.01696   0.00459
 0 O 3dx2-y2  -0.00617   0.00475  -0.01129  -0.00135   0.00947
  1 H 1s       0.04708   0.17694  -0.14521   0.08371 

In [110]:
fock_ao = mf.get_fock()
fock_pio_a = reduce(numpy.dot, [c_pio_a.T, fock_ao, c_pio_a])
dump_rec(stdout, fock_pio_a)

        #0        #1        #2        #3        #4       
0     -11.08093  -2.38395   0.74270   9.41881  -1.39817
1      -2.38395  -1.23596   0.33485   2.40497  -0.66313
2       0.74270   0.33485  -0.49114  -0.64387  -0.04174
3       9.41881   2.40497  -0.64387  -8.21238   1.30265
4      -1.39817  -0.66313  -0.04174   1.30265  -0.69713
5       1.43332  -0.21554   0.00584  -1.90143   0.05499
6      -1.78120  -0.15470   0.40812   1.25643  -0.51532
        #5        #6       
0       1.43332  -1.78120
1      -0.21554  -0.15470
2       0.00584   0.40812
3      -1.90143   1.25643
4       0.05499  -0.51532
5      -0.69134   0.30887
6       0.30887  -0.22574


In [109]:
c_ao_lo_a = c_ao_lo[:,idx_a]
c_pio_a   = reduce(numpy.dot, [c_ao_lo_a, ua.T])
npio = c_ao_lo_a.shape[1]

for pp in range(npio):
    tools.cubegen.orbital(mol, f'./cube/h2o_a_{pp}_{s[pp]:4.2f}_t.cube', c_pio_a[:,pp])

c_ao_lo_b = c_ao_lo[:,idx_b]
c_pio_b   = reduce(numpy.dot, [c_ao_lo_b, ub.T])
npio = c_ao_lo_b.shape[1]

for pp in range(npio):
    tools.cubegen.orbital(mol, f'./cube/h2o_b_{pp}_{s[pp]:4.2f}_t.cube', c_pio_b[:,pp])